# Understanding Lego sets popularity

Now let's now move on to the competition and challenge.

## 📖 Background
You recently applied to work as a data analyst intern at the famous Lego Group in Denmark. As part of the job interview process, you received the following take-home assignment:

You are asked to use the provided dataset to understand the popularity of different Lego sets and themes. The idea is to become familiarized with the data to be ready for an interview with a business stakeholder.

<img src="http://oyster.ignimgs.com/wordpress/stg.ign.com/2014/02/LEGOHeader2.jpg">

## 💾 The data

#### You received access to a database with the following tables. You can also see above a visualization of how the tables are related to each other. ([source](https://rebrickable.com/downloads)):

#### inventory_parts
- "inventory_id" - id of the inventory the part is in (as in the inventories table)
- "part_num" - unique id for the part (as in the parts table)
- "color_id" - id of the color
- "quantity" - the number of copies of the part included in the set
- "is_spare" - whether or not it is a spare part

#### parts
- "part_num" - unique id for the part (as in the inventory_parts table)
- "name" - name of the part
- "part_cat_id" - part category id (as in part_catagories table)

#### part_categories
- "id" - part category id (as in parts table)
- "name" - name of the category the part belongs to

#### colors
- "id" - id of the color (as in inventory_parts table)
- "name" - color name
- "rgb" - rgb code of the color
- "is_trans" - whether or not the part is transparent/translucent

#### inventories
- "id" - id of the inventory the part is in (as in the inventory_sets and inventory_parts tables)
- "version" - version number
- "set_num" - set number (as in sets table)

#### inventory_sets
- "inventory_id" - id of the inventory the part is in (as in the inventories table)
- "set_num" - set number (as in sets table)
- "quantity" - the quantity of sets included

#### sets
- "set_num" - unique set id (as in inventory_sets and inventories tables)
- "name" - the name of the set
- "year" - the year the set was published
- "theme_id" - the id of the theme the set belongs to (as in themes table)
- num-parts - the number of parts in the set

#### themes
- "id" - the id of the theme (as in the sets table)
- "name" - the name of the theme
- "parent_id" - the id of the larger theme, if there is one


***Acknowledgments**: Rebrickable.com*

![erd](data/lego_erd.png)

## 💪 Challenge
Create a report to summarize your findings. Include:

1. What is the average number of Lego sets released per year?
2. What is the average number of Lego parts per year?
3. Create a visualization for item 2.
4. What are the 5 most popular colors used in Lego parts?
5. [Optional] What proportion of Lego parts are transparent?
6. [Optional] What are the 5 rarest lego bricks?
7. Summarize your findings.

In [5]:
-- 1. What is the average number of Lego sets released per year?

SELECT avg(count) as lego_sets_released_per_year
    FROM (
        SELECT COUNT(d.set_num) as Count,
        d.set_num,
        d.year
        FROM inventory_parts a
        INNER JOIN inventories b 
            ON a.inventory_id = b.id
        INNER JOIN inventory_sets c
            ON b.id = c.inventory_id
        INNER JOIN sets d
            ON b.set_num = d.set_num
        Group by d.year, d.set_num
    ) as counts

,lego_sets_released_per_year
0,8.470588


In [64]:
-- 2. What is the average number of Lego parts per year?
SELECT sum(count_partition)/count(distinct(year_partition)) as lego_parts_per_year
    FROM (
        SELECT  e.part_num,
                COUNT(e.part_num) OVER (PARTITION BY d.year
                                       ) as count_partition,
                d.year as year_partition
                FROM inventory_parts a
                INNER JOIN inventories b 
                    ON a.inventory_id = b.id
                INNER JOIN sets d
                    ON b.set_num = d.set_num
                INNER JOIN parts e 
                    ON a.part_num = e.part_num
                GROUP BY e.part_num, d.year
        ) as counts

,lego_parts_per_year
0,2.460838e+06


In [6]:
WITH table_count_partition AS (

    SELECT  e.part_num,
                    COUNT(e.part_num) OVER (PARTITION BY d.year
                                           ) as count,
                    d.year
                    FROM inventory_parts a
                    INNER JOIN inventories b 
                        ON a.inventory_id = b.id
                    INNER JOIN sets d
                        ON b.set_num = d.set_num
                    INNER JOIN parts e 
                        ON a.part_num = e.part_num
                    GROUP BY e.part_num, d.year),
                    
table_sum_group_by AS (

    SELECT SUM(count) as sum_count,
           year
    from table_count_partition
    GROUP BY year)
    

SELECT (SUM(sum_count)/COUNT(year)
       ) as avg_lego_parts_per_year,
        year
FROM table_sum_group_by
GROUP BY year;
                    
                

,avg_lego_parts_per_year,year
0,36.0,1950
1,36.0,1953
2,729.0,1954
3,7396.0,1955
4,2209.0,1956
...,...,...
61,14333796.0,2013
62,13883076.0,2014
63,17355556.0,2015
64,18464209.0,2016


In [23]:
-- 4. What are the 5 most popular colors used in Lego parts?

WITH table_count_colors AS (
    SELECT  b.name,
            COUNT(a.part_num) OVER (PARTITION BY b.name
                               ) as count_name_over_partition
    FROM inventory_parts a
    INNER JOIN colors b 
    ON a.inventory_id = b.id
    GROUP BY 
    a.part_num,
    b.name),
    
table_sum_colors AS (
SELECT SUM(count_name_over_partition) as sum_total_colors,
        name
FROM table_count_colors
GROUP BY name)    

    
SELECT RANK () OVER (
                     ORDER BY sum_total_colors DESC
                     ) AS color_rank,
                     name
FROM table_sum_colors
LIMIT 5;


,color_rank,name
0,1,Royal Blue
1,2,Medium Blue
2,3,Very Light Bluish Gray
3,4,Sand Blue
4,4,Blue-Violet


In [5]:
-- 5. [Optional] What proportion of Lego parts are transparent?

WITH table_count_colors AS (
    SELECT  b.name,
            b.is_trans,
            COUNT(a.part_num) OVER (PARTITION BY b.name
                               ) AS count_name_over_partition
    FROM inventory_parts a
    INNER JOIN colors b 
    ON a.inventory_id = b.id
    GROUP BY 
    a.part_num,
    b.is_trans,
    b.name),
    
table_sum_colors AS (
    SELECT SUM(count_name_over_partition
              ) AS sum_total_colors,
            name,
            is_trans
    FROM table_count_colors
    GROUP BY 
    name,
    is_trans)


SELECT (SUM(sum_total_colors
          )/(SELECT SUM(sum_total_colors) 
             FROM table_sum_colors) * 100
       ) AS proportion_trans_parts
FROM table_sum_colors
WHERE is_trans = True;

,proportion_trans_parts
0,8.17635


In [22]:
SELECT 
a.part_num,
 b.name
FROM inventory_parts a
INNER JOIN parts b 
ON a.part_num = b.part_num
WHERE b.name LIKE '%Brick'
LIMIT 10;


,part_num,name
0,2577,Brick Round Corner 4 x 4 Full Brick
1,fabah4-hinge,"Fabuland Car Roof, Flippable - Hinge Brick"
2,2577,Brick Round Corner 4 x 4 Full Brick
3,2577,Brick Round Corner 4 x 4 Full Brick
4,2577,Brick Round Corner 4 x 4 Full Brick
5,2577,Brick Round Corner 4 x 4 Full Brick
6,2577,Brick Round Corner 4 x 4 Full Brick
7,2577,Brick Round Corner 4 x 4 Full Brick
8,884a,Mindstorms RCX 1.0 with Power Jack - Complete ...
9,2577,Brick Round Corner 4 x 4 Full Brick


In [23]:
-- 6. [Optional] What are the 5 rarest lego bricks?

SELECT RANK () OVER (ORDER BY count_bricks ASC
                 ) AS rarest_bricks,
                  name
FROM (
        SELECT COUNT(a.part_num
                    ) AS count_bricks,
                    a.part_num,
                    b.name
        FROM inventory_parts a
        INNER JOIN parts b 
        ON a.part_num = b.part_num
        WHERE b.name LIKE '%Brick'
        GROUP BY b.name, a.part_num) t
        LIMIT 5;
   
         


,rarest_bricks,name
0,1,Mindstorms Scout - Complete Brick
1,1,Mindstorms RCX 1.0 without Power Jack (from Mi...
2,3,Mindstorms EV3 - Complete Brick
3,3,"Fabuland Car Roof, Flippable - Hinge Brick"
4,3,Mindstorms RCX 1.0 with Power Jack - Complete ...


**7. Summarize your findings.**
1. What is the average number of Lego sets released per year? **8.4705882353**
2. What is the average number of Lego parts per year? **2460837.863636364**
4. What are the 5 most popular colors used in Lego parts?
	1. Royal Blue
	2. Medium Blue
	3. Very Light Bluish Gray
    4. Sand Blue
    4. Blue-Violet
5. [Optional] What proportion of Lego parts are transparent? **8.176350326**
6. [Optional] What are the 5 rarest lego bricks? 
	1. Mindstorms Scout - Complete Brick
	2. Mindstorms RCX 1.0 without Power Jack (from Mindstorms 1.5 sets) - Complete Brick
	3. Mindstorms EV3 - Complete Brick
    3. Fabuland Car Roof, Flippable - Hinge Brick
   	3. Mindstorms RCX 1.0 with Power Jack - Complete Brick